In [87]:
from catboost import CatBoostClassifier,Pool
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np 
import matplotlib.pyplot as plt

In [88]:
data = pd.read_csv("./csv/training_data.csv")
data

,Gallstone Status,Age,Gender,Comorbidity,Coronary Artery Disease (CAD),Hypothyroidism,Hyperlipidemia,Diabetes Mellitus (DM),Height,Visceral Fat Rating (VFR),Hepatic Fat Accumulation (HFA)
0,0,50,0,0,0,0,0,0,185,9,0
1,0,47,0,1,0,0,0,0,176,15,0
2,0,61,0,0,0,0,0,0,171,15,0
3,0,41,0,0,0,0,0,0,168,6,1
4,0,42,0,0,0,0,0,0,178,8,2
...,...,...,...,...,...,...,...,...,...,...,...
314,1,49,0,0,0,0,0,0,178,11,2
315,1,31,1,0,0,0,0,0,157,2,0
316,1,58,0,0,0,0,0,0,172,16,2
317,1,37,1,0,0,0,0,0,177,8,3


In [89]:
selected_features_name = [
    "Hepatic Fat Accumulation (HFA)",
    "Visceral Fat Rating (VFR)",
    "Height",
    "Comorbidity",
    "Gender"
]

In [90]:
X = data[selected_features_name]
y = data['Gallstone Status']

In [ ]:
n_splits = 5 
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)


fold_accuracies = []
fold_precisions = []
fold_recalls = []
fold_f1_scores = []
fold_roc_auc_scores = []

print(f"Début de la validation croisée avec {n_splits} folds...")

for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    print(f"\n--- Fold {fold + 1}/{n_splits} ---")

    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    train_pool = Pool(X_train, y_train)
    val_pool = Pool(X_val, y_val)

    model = CatBoostClassifier(
        iterations=100,              
        learning_rate=0.1,          
        depth=6,                     
        loss_function='Logloss',     
        eval_metric='AUC',           
        random_seed=42,
        verbose=0,                   
        early_stopping_rounds=10     
    )


    print("Entraînement du modèle...")
    model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=10)

    y_pred_proba = model.predict_proba(X_val)[:, 1] 
    y_pred = model.predict(X_val)                   

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, zero_division=0) 
    recall = recall_score(y_val, y_pred, zero_division=0)
    f1 = f1_score(y_val, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_val, y_pred_proba)

    print(f"Accuracy du Fold {fold + 1}: {accuracy}")
    print(f"Precision du Fold {fold + 1}: {precision}")
    print(f"Recall du Fold {fold + 1}: {recall}")
    print(f"F1-Score du Fold {fold + 1}: {f1:}")
    print(f"ROC AUC du Fold {fold + 1}: {roc_auc:}")

    # Stockage des métriques
    fold_accuracies.append(accuracy)
    fold_precisions.append(precision)
    fold_recalls.append(recall)
    fold_f1_scores.append(f1)
    fold_roc_auc_scores.append(roc_auc)

print("\n--- Résultats de la Validation Croisée ---\n")
print(f"Accuracy moyenne: {np.mean(fold_accuracies):.4f} ,  Ecart_type( {np.std(fold_accuracies):.4f})")
print(f"Precision moyenne: {np.mean(fold_precisions):.4f} ,  Ecart_type({np.std(fold_precisions):.4f})")
print(f"Recall moyen: {np.mean(fold_recalls):.4f} ,  Ecart_type({np.std(fold_recalls):.4f})")
print(f"F1-Score moyen: {np.mean(fold_f1_scores):.4f} ,  Ecart_type({np.std(fold_f1_scores):.4f})")
print(f"ROC AUC moyen: {np.mean(fold_roc_auc_scores):.4f} ,  Ecart_type({np.std(fold_roc_auc_scores):.4f})")


Début de la validation croisée avec 5 folds...

--- Fold 1/5 ---
Entraînement du modèle...
Accuracy du Fold 1: 0.7656
Precision du Fold 1: 0.7667
Recall du Fold 1: 0.7419
F1-Score du Fold 1: 0.7541
ROC AUC du Fold 1: 0.8021

--- Fold 2/5 ---
Entraînement du modèle...
Accuracy du Fold 2: 0.5625
Precision du Fold 2: 0.5667
Recall du Fold 2: 0.5312
F1-Score du Fold 2: 0.5484
ROC AUC du Fold 2: 0.5649

--- Fold 3/5 ---
Entraînement du modèle...
Accuracy du Fold 3: 0.6094
Precision du Fold 3: 0.6129
Recall du Fold 3: 0.5938
F1-Score du Fold 3: 0.6032
ROC AUC du Fold 3: 0.5640

--- Fold 4/5 ---
Entraînement du modèle...
Accuracy du Fold 4: 0.6719
Precision du Fold 4: 0.6486
Recall du Fold 4: 0.7500
F1-Score du Fold 4: 0.6957
ROC AUC du Fold 4: 0.7334

--- Fold 5/5 ---
Entraînement du modèle...
Accuracy du Fold 5: 0.5556
Precision du Fold 5: 0.5882
Recall du Fold 5: 0.3226
F1-Score du Fold 5: 0.4167
ROC AUC du Fold 5: 0.6336

--- Résultats de la Validation Croisée ---

Accuracy moyenne: 0.633

## Interprétation des Résultats de la Validation Croisée

L'analyse des métriques obtenues via la validation croisée fournit un aperçu de la performance et de la stabilité du modèle de classification binaire.

---

### Résumé des Performances du Modèle

| Métrique    | Moyenne | Écart-type | Interprétation Clé |
| :---------- | :------ | :--------- | :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Précision** | 0.6366  | 0.0705     | Le modèle est **correct 63.66% du temps** quand il prédit une instance positive. La variabilité (0.0705) est modérée, indiquant une performance relativement stable sur ce critère. |
| **Rappel** | 0.5879  | 0.1573     | Le modèle ne détecte que **58.79% des vrais cas positifs**. L'écart-type élevé (0.1573) est notable, suggérant une **forte inconstance** dans sa capacité à identifier les positifs à travers les différents jeux de données. |
| **F1-Score** | 0.6036  | 0.1176     | Représentant un équilibre entre Précision et Rappel, le F1-Score moyen de **60.36%** est modéré, soulignant une performance globale correcte mais avec une marge d'amélioration. Son écart-type élevé reflète la variabilité du Rappel. |
| **ROC AUC** | 0.6596  | 0.0944     | Avec un score de **0.6596**, le modèle a une **capacité de distinction modérée** entre les classes positives et négatives. Il est mieux que le hasard (0.5), mais loin d'une séparation parfaite (1.0). L'écart-type indique une variabilité notable dans cette capacité. |

---

Le modèle montre une **performance globale modérée**, avec des scores qui, bien que supérieurs au hasard, révèlent un potentiel d'amélioration significatif. Le point le plus critique est la **faible et très inconsistante capacité du modèle à identifier les cas positifs (Rappel)**. Cette variabilité, associée à un Recall moyen bas, suggère que le modèle pourrait avoir du mal avec des classes déséquilibrées ou qu'il nécessite une optimisation plus poussée de ses hyperparamètres ou des efforts en ingénierie des caractéristiques pour améliorer sa robustesse et sa précision, particulièrement pour la détection des vrais positifs.